# Big Red Soundscape
Notebook Playground 2  
_by Ahmed Sultan and Carly Hu, for Cornell Data Journal_

In [61]:
import json
import math

import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
import seaborn as sns

In [62]:
# update access keys here for Spotify API here :) 
client_id = 'f7661e34c0924317a8b928c0f977e6a7'
client_secret = 'd068ec3f0a98490e9c32be820916e934'

# sp is how we interface with the Spotify API
# read the docs here: https://spotipy.readthedocs.io/en/2.22.1/
sp = spotipy.Spotify(client_credentials_manager=\
                     SpotifyClientCredentials(client_id, client_secret))

<h1> Let's load in our survey data! </h1>

In [72]:
students_df = pd.read_csv('datacollection_042623.csv')
students_df.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q8_3,Q8_4,Q9,Q10,Q11_1,Q11_2,Q11_3,Q11_4,Q11_5,Q12
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,How do you perceive yourself? - Introverted (0...,How do you perceive yourself? - Logical (0) vs...,Link your Spotify profile here—this provides u...,Link a Spotify playlist that captures your mus...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you would like the chance to win a $15 gift...
1,"{""ImportId"":""startDate"",""timeZone"":""America/De...","{""ImportId"":""endDate"",""timeZone"":""America/Denv...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID18_3""}","{""ImportId"":""QID18_4""}","{""ImportId"":""QID19_TEXT""}","{""ImportId"":""QID20_TEXT""}","{""ImportId"":""QID21_1""}","{""ImportId"":""QID21_2""}","{""ImportId"":""QID21_3""}","{""ImportId"":""QID21_4""}","{""ImportId"":""QID21_5""}","{""ImportId"":""QID24_TEXT""}"
2,2023-04-12 15:22:47,2023-04-12 15:24:16,IP Address,*******,100,89,True,2023-04-12 15:24:17,R_2YtbSvySpz2s87b,*******,...,4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-04-12 15:22:48,2023-04-12 15:26:13,IP Address,*******,100,205,True,2023-04-12 15:26:14,R_24ixQUbGn1AHnjs,*******,...,7,4,https://open.spotify.com/user/6cjzkudbvmwfe963...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN
4,2023-04-12 15:22:51,2023-04-12 15:26:40,IP Address,*******,100,229,True,2023-04-12 15:26:41,R_aWSgue6VU9HyHMR,*******,...,8,5,https://open.spotify.com/user/vqlx3sn88fh9295c...,https://open.spotify.com/playlist/6huxxOvKkbRu...,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
# get rid of some silly columns
students_df = students_df.drop(columns=['StartDate', 'EndDate',
                                        'Status','IPAddress','Duration (in seconds)','RecordedDate',
                                        'ResponseId','RecipientLastName','RecipientFirstName',
                                        'RecipientEmail', 'ExternalReference','LocationLatitude', 
                                        'LocationLongitude','DistributionChannel', 'UserLanguage'])
students_df.head() # so many NaNs oh god

,Progress,Finished,Q1,Q2,Q3,Q4,Q7,Q5,Q6,Q6.1,...,Q8_3,Q8_4,Q9,Q10,Q11_1,Q11_2,Q11_3,Q11_4,Q11_5,Q12
0,Progress,Finished,What college are you primarily affiliated with?,What is your (primary) major?,What year are you?,Gender,Extracurricular involvements (select all that ...,Are you an international or domestic (U.S.) st...,What country are you from?,What state are you from?,...,How do you perceive yourself? - Introverted (0...,How do you perceive yourself? - Logical (0) vs...,Link your Spotify profile here—this provides u...,Link a Spotify playlist that captures your mus...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you would like the chance to win a $15 gift...
1,"{""ImportId"":""progress""}","{""ImportId"":""finished""}","{""ImportId"":""QID5""}","{""ImportId"":""QID14""}","{""ImportId"":""QID8""}","{""ImportId"":""QID10""}","{""ImportId"":""QID17""}","{""ImportId"":""QID11""}","{""ImportId"":""QID15""}","{""ImportId"":""QID16""}",...,"{""ImportId"":""QID18_3""}","{""ImportId"":""QID18_4""}","{""ImportId"":""QID19_TEXT""}","{""ImportId"":""QID20_TEXT""}","{""ImportId"":""QID21_1""}","{""ImportId"":""QID21_2""}","{""ImportId"":""QID21_3""}","{""ImportId"":""QID21_4""}","{""ImportId"":""QID21_5""}","{""ImportId"":""QID24_TEXT""}"
2,100,True,(CALS) College of Agriculture and Life Sciences,Biometry and Statistics,"Undergraduate, 4th year (2023)",Male,"Project team,Other club/team",Domestic,NaN,Florida,...,4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100,True,(AS) College of Arts and Sciences,Computer Science,"Undergraduate, 1st year (2026)",Male,Other club/team,International,Korea South,NaN,...,7,4,https://open.spotify.com/user/6cjzkudbvmwfe963...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN
4,100,True,(AS) College of Arts and Sciences,Government,"Undergraduate, 1st year (2026)",Female,Professional club,International,Indonesia,NaN,...,8,5,https://open.spotify.com/user/vqlx3sn88fh9295c...,https://open.spotify.com/playlist/6huxxOvKkbRu...,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
# drop anyone who didn't link a profile properly

students_df = students_df.dropna(0, subset=['Q9'])
students_df = students_df[students_df["Q9"].str.contains('https://open.spotify.com/user')]

# reset index 
students_df = students_df.reset_index()

# ok what do we have now
print('Number of linked spotify profiles: ' + str(students_df.shape[0]-1))
students_df.head()

Number of linked spotify profiles: 45


/var/folders/qg/5ts3y3kd41l7546h43d5mdy00000gn/T/ipykernel_44418/1025512729.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  students_df = students_df.dropna(0, subset=['Q9'])


,index,Progress,Finished,Q1,Q2,Q3,Q4,Q7,Q5,Q6,...,Q8_3,Q8_4,Q9,Q10,Q11_1,Q11_2,Q11_3,Q11_4,Q11_5,Q12
0,0,Progress,Finished,What college are you primarily affiliated with?,What is your (primary) major?,What year are you?,Gender,Extracurricular involvements (select all that ...,Are you an international or domestic (U.S.) st...,What country are you from?,...,How do you perceive yourself? - Introverted (0...,How do you perceive yourself? - Logical (0) vs...,Link your Spotify profile here—this provides u...,Link a Spotify playlist that captures your mus...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you would like the chance to win a $15 gift...
1,3,100,True,(AS) College of Arts and Sciences,Computer Science,"Undergraduate, 1st year (2026)",Male,Other club/team,International,Korea South,...,7,4,https://open.spotify.com/user/6cjzkudbvmwfe963...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN
2,4,100,True,(AS) College of Arts and Sciences,Government,"Undergraduate, 1st year (2026)",Female,Professional club,International,Indonesia,...,8,5,https://open.spotify.com/user/vqlx3sn88fh9295c...,https://open.spotify.com/playlist/6huxxOvKkbRu...,NaN,NaN,NaN,NaN,NaN,NaN
3,7,100,True,(ENG) College of Engineering,Cognitive Science,"Undergraduate, 1st year (2026)",Male,"Project team,Other club/team",Domestic,NaN,...,4,4,https://open.spotify.com/user/m3l6ktgr5im6yorm...,https://open.spotify.com/playlist/3Tp8L5xWN4OQ...,https://open.spotify.com/playlist/4ueOZx7jYJC1...,NaN,NaN,NaN,NaN,NaN
4,8,100,True,(ILR) School of Industrial and Labor Relations,Industrial and Labor Relations,"Undergraduate, 3rd year (2024)",Female,"Professional club,Other club/team",Domestic,NaN,...,5,7,https://open.spotify.com/user/t4llwr7d08bca91r...,https://open.spotify.com/playlist/76R0Tg5FCTrb...,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
# let's get a list of spotify usernames

usernames = students_df['Q9']

stripped_usernames = []

for i in usernames:
    i=i.replace("https://open.spotify.com/user/", "")
    i=i.split('?si',1)[0]
    stripped_usernames.append(i)

    i =+ 1

print(len(stripped_usernames))
stripped_usernames

46


['Link your Spotify profile here—this provides us with access to your public playlists. \n(ex. your_username)',
 '6cjzkudbvmwfe963clvemzwkj',
 'vqlx3sn88fh9295cpn4z7toip',
 'm3l6ktgr5im6yormbf36etw78',
 't4llwr7d08bca91rqn1ro2aau',
 'ejlity',
 'aaa9rnpre89bf2i0tx7ihyem0',
 'deadpan.typicalnerdbird1492',
 'fluttershyfatima',
 '1283949946',
 '31m3onadeh4wxucgckv3fhcyeoqu',
 'sir.frances.drake',
 'seemslikehope',
 'tiffxny.lee',
 'sharktiger',
 'bn8wfbeyvujlinqcmoj41nymr',
 'owllson',
 'alexabarton16',
 'mtaakmfvi2rkrcrkbg3qsv1b0',
 '31tczmefxwht47ndcx5rnqqztaaa',
 'alayna.earl',
 'n8hn1814',
 'd4iyjnkubkj5j1rmdr6rxdp1y',
 'bad_mug',
 'sup.arna',
 'kobitravels',
 'jessica20022',
 'o035tufj1j2to59df208ozh5k',
 'meepsymo',
 'vbvb101',
 'dihbuj006e36pz2kkpeiyr9it',
 'clarebelman',
 '1re58jm2ppu8vkex6p2lqkyvt',
 'andreacmauri',
 'olivia-1212',
 '12124545665',
 '12139305999',
 'jhkmaggie',
 'eirian_h',
 'lenatp808',
 'ayamtume',
 '313sqjo7imdsqz2sub73bwrnsdlm',
 'whc30ixinkb9qw5xkfcn6pg5z',
 '

In [89]:
students_df['username'] = stripped_usernames # adding username column
students_df.head() # READY FOR ANALYSIS WOOOO

,index,Progress,Finished,Q1,Q2,Q3,Q4,Q7,Q5,Q6,...,Q8_4,Q9,Q10,Q11_1,Q11_2,Q11_3,Q11_4,Q11_5,Q12,username
0,0,Progress,Finished,What college are you primarily affiliated with?,What is your (primary) major?,What year are you?,Gender,Extracurricular involvements (select all that ...,Are you an international or domestic (U.S.) st...,What country are you from?,...,How do you perceive yourself? - Logical (0) vs...,Link your Spotify profile here—this provides u...,Link a Spotify playlist that captures your mus...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you have Spotify playlists for the followin...,If you would like the chance to win a $15 gift...,Link your Spotify profile here—this provides u...
1,3,100,True,(AS) College of Arts and Sciences,Computer Science,"Undergraduate, 1st year (2026)",Male,Other club/team,International,Korea South,...,4,https://open.spotify.com/user/6cjzkudbvmwfe963...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,NaN,https://open.spotify.com/playlist/37HUliOloWGE...,https://open.spotify.com/playlist/5Kwx1SSB0pol...,NaN,6cjzkudbvmwfe963clvemzwkj
2,4,100,True,(AS) College of Arts and Sciences,Government,"Undergraduate, 1st year (2026)",Female,Professional club,International,Indonesia,...,5,https://open.spotify.com/user/vqlx3sn88fh9295c...,https://open.spotify.com/playlist/6huxxOvKkbRu...,NaN,NaN,NaN,NaN,NaN,NaN,vqlx3sn88fh9295cpn4z7toip
3,7,100,True,(ENG) College of Engineering,Cognitive Science,"Undergraduate, 1st year (2026)",Male,"Project team,Other club/team",Domestic,NaN,...,4,https://open.spotify.com/user/m3l6ktgr5im6yorm...,https://open.spotify.com/playlist/3Tp8L5xWN4OQ...,https://open.spotify.com/playlist/4ueOZx7jYJC1...,NaN,NaN,NaN,NaN,NaN,m3l6ktgr5im6yormbf36etw78
4,8,100,True,(ILR) School of Industrial and Labor Relations,Industrial and Labor Relations,"Undergraduate, 3rd year (2024)",Female,"Professional club,Other club/team",Domestic,NaN,...,7,https://open.spotify.com/user/t4llwr7d08bca91r...,https://open.spotify.com/playlist/76R0Tg5FCTrb...,NaN,NaN,NaN,NaN,NaN,NaN,t4llwr7d08bca91rqn1ro2aau


In [90]:
# HELPER FUNCTIONS DUMP
def user_playlist_tracks_full(user, playlist_id=None, fields=None):
    """ Get full details of the tracks of a playlist owned by a user.
        https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlists-tracks/

        Parameters:
            - user - the id of the user
            - playlist_id - the id of the playlist
            - fields - which fields to return
    """

    # first run through also retrieves total no of songs in library
    response = sp.user_playlist_tracks(user, playlist_id, fields=None, limit=100)
    results = response["items"]

    # subsequently runs until it hits the user-defined limit or has read all songs in the library
    while len(results) < response["total"]:
        response = sp.user_playlist_tracks(
            user, playlist_id, fields=fields, limit=100, offset=len(results)
        )
        results.extend(response["items"])
    
    return results

def get_tracks_full(songs):
    results = []
    while len(results) < len(songs):
        response = (sp.tracks(songs[len(results):len(results) + 50]))['tracks']
        results.extend(response)
    return results

def get_audio_features_full(songs):
    results = []
    while len(results) < len(songs):
        response = sp.audio_features(songs[len(results):len(results) + 100])
        results.extend(response)
    return results

<h1> Time to process each user in a huge for-loop </h1>

In [ ]:
all_averages = []

for selected_user in stripped_usernames[1:46]:
    playlists = []
    LIMIT = 20

    query = sp.user_playlists(selected_user, limit=LIMIT)

    n = 0

    while len(query['items']) != 0:
        n += LIMIT
        for i in query['items']:
            playlists.append({'title': i['name'],'id': i['id'], 'length': i['tracks']['total']})
        if LIMIT == len(query['items']):
            query = sp.user_playlists(selected_user, limit=LIMIT, offset=n)
        else:
            break
            
    songs = []

    for p in playlists:

        results = sp.playlist(p['id'])

        extended_results = user_playlist_tracks_full(user = selected_user, playlist_id = results['id']) # more than 100

        for r in extended_results:
            songs.append(r['track']['id'])
    
    songs_meta = get_tracks_full(songs)

    song_info = {'id': [], 'title': [], 'artist': [], 'album': [], 
                 'explicit': [], 'popularity': []}

    for i in songs_meta:
        # append id to id column
        song_info['id'].append(i['id'])

        # append song title to title column
        song_info['title'].append(i['name'])

        # append artists to artist column
        s = ', '
        artists = s.join([name['name'] for name \
                         in i['artists']])
        song_info['artist'].append(artists)

        # append album title to album column
        song_info['album'].append(i['album']['name'])

        # append explicit info to explicit column
        song_info['explicit'].append(i['explicit'])

        # append popularity info to popularity column
        song_info['popularity'].append(i['popularity'])

    # convert song_info dictionary -> Pandas DataFrame
    song_info_df = pd.DataFrame.from_dict(song_info)

    song_info_df = song_info_df.drop_duplicates(subset=['id'])
    song_info_df = song_info_df.dropna()
    
    features_df = pd.DataFrame.from_dict(get_audio_features_full(songs))
    features_df = features_df.drop_duplicates(subset=['id'])
    features_df = features_df.dropna()
    
    df = song_info_df.merge(features_df)
    
    features = ['popularity','danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

    feature_averages = []

    for feature in features:
        avg =  df[feature].mean()
        feature_averages.append(avg)
   
    all_averages.append(feature_averages)

print(all_averages)

In [ ]:
user_stats_df = pd.DataFrame(all_averages,
                  columns = ['popularity','danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'])

user_stats_df